# Interest Rate Vanilla Swaps

This notebook demonstrates how to access and use the **IR Swaps** functionalities within the **LSEG Financial Analytics SDK**.

Interest rate swaps involve exchanging one stream of interest payments for another to manage exposure to interest rate fluctuations or achieve better borrowing terms. Typically, this swap involves converting fixed-rate payments to floating rates or vice versa.

**You will be able to:**
- Define an OTC IR Swap
- Evaluate the swap
- Display cashflows and sensibilities
- Override the spread of the floating leg
- Display the available indexes for the floating leg

## Imports
Import the following necessary modules:
This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import ir_swaps as irs
from lseg_analytics.pricing.common import AdjustableDate, RelativeAdjustableDate, ReferenceDate, FrequencyEnum
import lseg_analytics.pricing.reference_data.floating_rate_indices as fri
import pandas as pd
from IPython.display import display

#### Define the Fixed Leg

In [2]:
# All the attributes below are the required ones for the class "InterestRateLegDefinition".
irs_first_leg = irs.InterestRateLegDefinition(
    rate = irs.FixedRateDefinition(
        rate = irs.Rate(
            value = 3,
            unit = irs.UnitEnum.PERCENTAGE
        )
    ),
    interest_periods = irs.ScheduleDefinition(
        start_date = AdjustableDate(  # Adjustable Date could be modified depending on the conventions
            date = "2025-01-01"
        ),
        end_date = RelativeAdjustableDate( # Relative Adjustable Date could be modified depending on the conventions and tenor
            tenor = "1Y", 
            reference_date = ReferenceDate.START_DATE
        ),
        frequency = FrequencyEnum.QUARTERLY
    ),
    principal = irs.PrincipalDefinition(amount = 1E6, currency = 'USD'),
    payer = irs.PartyEnum.PARTY1,
    receiver = irs.PartyEnum.PARTY2
)

#### Define the Floating Leg

Before defining the leg, the code below explains how identify an index name in order to use it as a floating rate. More details are available at the end of the notebook.

In [3]:
# avaibale IR curves for USD
print(fri.search(tags=["currency:USD"]))

[{'type': 'FloatingRateIndex', 'id': '67b1858d-4113-45d7-b401-8de42bd3fec0', 'location': {'space': 'LSEG', 'name': 'USD_BMA_FIX_7D'}, 'description': {'summary': 'SIFMA Fix', 'tags': ['currency:USD', 'indexTenor:7D', 'sourceLongName:Refinitiv', 'sourceShortName:RFTB']}}, {'type': 'FloatingRateIndex', 'id': '710fc04c-e793-44f4-a0f1-b133c8a37c87', 'location': {'space': 'LSEG', 'name': 'USD_FFER_ON'}, 'description': {'summary': 'Fed Funds Effective Rate', 'tags': ['currency:USD', 'indexTenor:ON', 'sourceLongName:Refinitiv', 'sourceShortName:RFTB']}}, {'type': 'FloatingRateIndex', 'id': '10a30cf1-034d-4d3d-8f88-b808cc71e15c', 'location': {'space': 'LSEG', 'name': 'USD_LIBOR_1M_IBA'}, 'description': {'summary': 'USD LIBOR (Ice Benchmark Administration)', 'tags': ['currency:USD', 'indexTenor:1M', 'sourceLongName:Ice Benchmark Administration', 'sourceShortName:IBA']}}, {'type': 'FloatingRateIndex', 'id': 'd8e2da70-7a3e-4f16-be4d-5f5b81766d95', 'location': {'space': 'LSEG', 'name': 'USD_LIBOR_1

In [4]:
# to get the correct index name, one need to use index space then the index name as below
sofr_index = [usd_index for usd_index in fri.search(tags=["currency:USD", "indexTenor:ON"]) if "SOFR" in usd_index.location.name][0]
sofr_index_name = sofr_index.location.space + "/" + sofr_index.location.name

In [5]:
irs_second_leg = irs.InterestRateLegDefinition(
    rate = irs.FloatingRateDefinition(
        index = sofr_index_name
    ),
    interest_periods = irs.ScheduleDefinition(
        start_date = AdjustableDate(  # Adjustable Date could be modified depending on the conventions
            date = "2025-01-01"
        ),
        end_date = RelativeAdjustableDate( # Relative Adjustable Date could be modified depending on the conventions and tenor
            tenor = "1Y", 
            reference_date = ReferenceDate.START_DATE
        ),
        frequency = FrequencyEnum.QUARTERLY
    ),
    principal = irs.PrincipalDefinition(amount = 1E6, currency = 'USD'),
    payer = irs.PartyEnum.PARTY2,
    receiver = irs.PartyEnum.PARTY1
)

#### Define and create the IR Swap

In [6]:
# Define the swap using the legs defined before
ir_swap_definition = irs.IrSwapDefinition(
    first_leg = irs_first_leg,
    second_leg = irs_second_leg
)

# Create the instrument from the definition
irs_instrument = irs.IrSwapDefinitionInstrument(
    definition = ir_swap_definition
)

# Instantiate the pricing parameters
pricing_params = irs.IrPricingParameters(
    valuation_date = "2025-07-18",
    report_currency = "USD"
)

#### Evaluate the Swap

In [7]:
#  Execute the calculation using the value() function
# The 'definitions' parameter accepts a list of instruments definitions for batch processing

# Execute the calculation using the value() function with error handling
try:
    response = irs.value(
        definitions=[irs_instrument],
        pricing_preferences = pricing_params
        
    )
    errors = [a.error for a in response.analytics if a.error]
    if errors:
        raise Exception(errors[0].message)
    print("IR Swap pricing execution completed")
except Exception as e:
    print(f"Price Calculation failed: {str(e)}")
    raise

IR Swap pricing execution completed


#### Key Sections in the `response`

 - **definitions**: Instument definitions set above. 

 - **pricingPreferences**: Valuation date.

 - **analytics**:
   - **description**: Instrument summary.
   - **valuation**: Valuation details.
   - **risk**: Sensitivities (duration, dv01 etc.)
   - **firstLeg**: description, valuation, risk and cashflows for the first leg
   - **secondLeg**: description, valuation, risk and cashflows for the second leg

### Valuation Details

In [8]:
valuation = response.analytics[0].valuation
print("Market value", valuation["marketValue"]["value"])
print("Accrued value", valuation["accrued"]["value"])
print("Clean market value", valuation["cleanMarketValue"]["value"])

Market value 6242.19605659611
Accrued value 636.572914538513
Clean market value 5605.62314205759


### Fixed CashFlows

In [9]:
fixed_cfs = response.analytics[0].first_leg["cashflows"]
start_dates = [cf['startDate'] for cf in fixed_cfs]
end_dates = [cf['endDate'] for cf in fixed_cfs]
fixed_rates = [cf['annualRate']['value'] for cf in fixed_cfs]
dfs = [cf['discountFactor'] for cf in fixed_cfs]
amounts = [cf['amount']['value'] for cf in fixed_cfs]

fixed_cfs_df = pd.DataFrame({"Start Dates": start_dates, "End Dates": end_dates, 
                                "Floating Rates": fixed_rates, "Discount Factors": dfs, "CashFlow Amounts": amounts})
display(fixed_cfs_df)

,Start Dates,End Dates,Floating Rates,Discount Factors,CashFlow Amounts
0,2025-01-02,2025-04-01,3.0,1.000000,-7416.666667
1,2025-04-01,2025-07-01,3.0,1.000000,-7583.333333
2,2025-07-01,2025-10-01,3.0,0.991035,-7666.666667
3,2025-10-01,2026-01-02,3.0,0.980613,-7750.000000


### Floating CashFlows Details

In [10]:
floating_cfs = response.analytics[0].second_leg["cashflows"]
start_dates = [cf['startDate'] for cf in floating_cfs]
end_dates = [cf['endDate'] for cf in floating_cfs]
floating_rates = [cf['annualRate']['value'] for cf in floating_cfs]
dfs = [cf['discountFactor'] for cf in floating_cfs]
amounts = [cf['amount']['value'] for cf in floating_cfs]

floating_cfs_df = pd.DataFrame({"Start Dates": start_dates, "End Dates": end_dates, 
                                "Floating Rates": floating_rates, "Discount Factors": dfs, "CashFlow Amounts": amounts})
display(floating_cfs_df)

,Start Dates,End Dates,Floating Rates,Discount Factors,CashFlow Amounts
0,2025-01-02,2025-04-01,4.351480,1.000000,10757.826606
1,2025-04-01,2025-07-01,4.354492,1.000000,11007.189199
2,2025-07-01,2025-10-01,4.350304,0.991035,11117.444179
3,2025-10-01,2026-01-02,4.114123,0.980613,10628.150883


### Sensitivities of the Swap

In [11]:
response.analytics[0].risk.duration.value
risks = response.analytics[0].risk
risk_df = pd.DataFrame({"Fields": list(risks.keys()), "Values": [risks[item]["value"] for item in risks]})
display(risk_df)

,Fields,Values
0,duration,-2.528505e-01
1,modifiedDuration,-4.392427e-01
2,benchmarkHedgeNotional,-2.085322e+06
3,annuity,-5.065897e+01
4,dv01,-4.374027e+01
5,pv01,-4.374027e+01
6,br01,0.000000e+00


### Override the spread within the Floating Leg

In [12]:
irs_second_leg.rate.spread_schedule = [irs.DatedRate(
    rate = irs.Rate(value = 5, unit = irs.UnitEnum.BASIS_POINT) # spread if 5 Bps
)]

In [13]:
# Reprice the swap to visualize the impact of the spread
try:
    response = irs.value(
        definitions=[irs_instrument],
        pricing_preferences = pricing_params
        
    )
    errors = [a.error for a in response.analytics if a.error]
    if errors:
        raise Exception(errors[0].message)
    print("IR Swap pricing execution completed")
except Exception as e:
    print(f"Price Calculation failed: {str(e)}")
    raise

IR Swap pricing execution completed


#### Comparison between the floating rates to visualize the impact of the spread

In [14]:
floating_rates_with_spread = [cf['annualRate']['value'] for cf in response.analytics[0].second_leg["cashflows"]]
compare_floating_rates_df = pd.DataFrame({"Floating Rates Without Spread": floating_rates,
                                          "Floating Rates Wiht 5bp Sperad": floating_rates_with_spread})
display(compare_floating_rates_df)

,Floating Rates Without Spread,Floating Rates Wiht 5bp Sperad
0,4.351480,4.402009
1,4.354492,4.405034
2,4.350304,4.400851
3,4.114123,4.164123


#### List of availble indexes for the floating leg

In [15]:
availabel_indexes = fri.search()
indexes = [item['location']['name'] for item in availabel_indexes]
currencies = [item['description']['tags'][0][9:] for item in availabel_indexes]
available_indexes_df = pd.DataFrame({"Index Name": indexes, "Currency": currencies})
display(available_indexes_df.head(10))

,Index Name,Currency
0,AED_AEIBOR_1M,AED
1,AED_AEIBOR_1Y,AED
2,AED_AEIBOR_3M,AED
3,AED_AEIBOR_6M,AED
4,AED_AEIBOR_SW,AED
5,ARS_BADLAR_1M,ARS
6,AUD_AONIA_ON_RBAA,AUD
7,AUD_AUDRBA_ON,AUD
8,AUD_BBSW_1M,AUD
9,AUD_BBSW_2M,AUD


To use this index name correctly, "LSEG/" must be added before as example "LSEG/USD_SOFR_ON"

Below, another tag must be added to select only USD indexes

In [16]:
availabel_indexes = fri.search(tags=["currency:USD"])
indexes = [item['location']['name'] for item in availabel_indexes]
currencies = [item['description']['tags'][0][9:] for item in availabel_indexes]
available_indexes_df = pd.DataFrame({"Index Name": indexes, "Currency": currencies})
display(available_indexes_df.head(10))

,Index Name,Currency
0,USD_BMA_FIX_7D,USD
1,USD_FFER_ON,USD
2,USD_LIBOR_1M_IBA,USD
3,USD_LIBOR_1Y_IBA,USD
4,USD_LIBOR_2M_IBA,USD
5,USD_LIBOR_3M_IBA,USD
6,USD_LIBOR_6M_IBA,USD
7,USD_LIBOR_ON_IBA,USD
8,USD_LIBOR_SW_IBA,USD
9,USD_SOFR_1M_CME,USD
